# WEO Economic Data Analysis & Recession Prediction

**Objective:** Load World Economic Outlook (WEO) data, clean and transform it, then use machine learning models to predict global recessions.

**Workflow:**
1. Data loading and cleaning
2. Feature engineering and recession flagging
3. Exploratory data analysis
4. Model training with full and reduced feature sets (comparing 13 vs 5 features)
5. Economy-specific analysis (Upper vs Lower economies with both feature sets)
6. Future predictions for all scenarios

**Models Used:** Logistic Regression, Random Forest, Gradient Boosting, Linear SVM, Decision Tree, and Ensemble

In [ ]:
#1)Multiclass classification target distribution - aanpassen -done
#2)Threshhold meerdere waardes vullen en dat in Overleaf plaatsen ter discussie -fail
#3)ROC, AUC laten zien -done
#4) Zoek verder op de auteur naar vergelijkbaar werk -wip
#5) Use Arima as baseline -wip
#6) Try neural network -wip

In [ ]:
# Core data manipulation and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# File handling
import csv
from pathlib import Path

# Machine learning - model selection and preprocessing
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA

# Machine learning - models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_curve, auc
from sklearn.utils.class_weight import compute_class_weight

# Machine learning - metrics
from sklearn.metrics import (
    classification_report, 
    accuracy_score, 
    precision_score, 
    recall_score, 
    f1_score, 
    roc_auc_score,
    confusion_matrix,
    ConfusionMatrixDisplay
)

# Imbalanced learning
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

# Optional XGBoost
try:
    from xgboost import XGBClassifier
    HAS_XGB = True
except ImportError:
    HAS_XGB = False

# Optional pycountry for continent mapping
try:
    import pycountry
    import pycountry_convert as pc
    HAS_PYCOUNTRY = True
except ImportError:
    HAS_PYCOUNTRY = False

pd.set_option("display.max_rows", 25)


# 1. Data Loading

In [ ]:
p = Path(r"data.csv")
if not p.exists():
    raise FileNotFoundError(p)

# Detect encoding and delimiter
encoding = "utf-8"
try:
    sample = p.read_text(encoding=encoding)[:8192]
except UnicodeDecodeError:
    encoding = "latin-1"
    sample = p.read_text(encoding=encoding)[:8192]

try:
    delim = csv.Sniffer().sniff(sample).delimiter
except Exception:
    delim = ","

df = pd.read_csv(p, sep=delim, encoding=encoding, low_memory=False, parse_dates=True)
print("Shape:", df.shape)
df.head()

In [ ]:
print(f"Number of columns: {df.shape[1]}")
print(f"Number of rows: {df.shape[0]}")
print("\nColumn names:", df.columns.tolist())

# 2. Data Cleaning & Transformation

## Filter to Selected Economic Indicators

In [ ]:
df.drop(columns=["WEO Country Code", "ISO", "Country/Series-specific Notes", "Subject Notes", 
                 "Units", "Scale", "Estimates Start After", "Subject Descriptor"], inplace=True)

codes = {
    # Core growth & external
    "NGDP_RPCH", "NGDPRPC", "PCPIPCH", "TX_RPCH", "TM_RPCH", "BCA_NGDPD",
    # Fiscal & debt aggregates
    "GGR_NGDP", "GGX_NGDP", "GGXWDN_NGDP", "GGXWDG_NGDP",
    # Savings & investment
    "NGSD_NGDP", "NID_NGDP",
    # Prices
    "PCPI", "LUR"
}

col = "WEO Subject Code"

if col not in df.columns:
    raise KeyError(f"Column {col!r} not found in dataframe")

df = df[df[col].astype(str).str.strip().isin(codes)].copy()
print("shape after filter:", df.shape)
df

## Data Reshaping: Wide to Long to Wide

In [ ]:
year_cols = df.columns[2:]

df[year_cols] = df[year_cols].replace({',': ''}, regex=True)
df[year_cols] = df[year_cols].apply(pd.to_numeric, errors="coerce")

df["Country"] = (
    df["Country"]
    .str.replace(" ", "_")
    .str.replace("'", "")
    .str.replace("-", "_")
)

df_long = df.melt(id_vars=["WEO Subject Code", "Country"],
                  var_name="Year", value_name="Value")

df_long["Year"] = df_long["Year"].astype(str).str.strip()
df_long = df_long[df_long["Year"].str.fullmatch(r"\d{4}")].copy()
df_long["Year"] = df_long["Year"].astype(int)

df_long["Value"] = (
    df_long["Value"].astype(str)
    .str.replace(",", "")
    .replace({"": None, "nan": None})
    .astype(float)
)

df_pivot = df_long.pivot_table(
    index=["Country", "Year"],
    columns="WEO Subject Code",
    values="Value",
    aggfunc="first"
).reset_index()

df_pivot.columns.name = None
df_pivot = df_pivot.set_index("Year")

df_pivot

# 3. Feature Engineering

## Add Recession Target Variable

In [ ]:
# --- Step 1: Ensure chronological order ---
df_pivot = df_pivot.sort_index()

# --- Step 2: Construct diagnostic flags (not stored in df) ---

# 1. GDP-based recession (two consecutive annual declines)
flag_gdp = (
    (df_pivot.groupby("Country")["NGDPRPC"].transform(lambda x: x.pct_change() < 0)) &
    (df_pivot.groupby("Country")["NGDPRPC"].transform(lambda x: x.pct_change().shift(-1) < 0))
).astype(int)

# 2. Investment collapse (sharp drop in investment)
flag_invest = (
    df_pivot.groupby("Country")["NID_NGDP"].transform(lambda x: x.diff() < -2)
).astype(int)

# 3. Savings decline (household/national savings falling)
flag_savings = (
    df_pivot.groupby("Country")["NGSD_NGDP"].transform(lambda x: x.diff() < -2)
).astype(int)

# 4. Trade shock (both exports and imports contracting)
flag_trade = (
    (df_pivot.groupby("Country")["TX_RPCH"].transform(lambda x: x < 0)) &
    (df_pivot.groupby("Country")["TM_RPCH"].transform(lambda x: x < 0))
).astype(int)

# 5. Inflation shock (stagflation: high inflation + negative growth)
flag_inflation = (
    (df_pivot.groupby("Country")["PCPIPCH"].transform(lambda x: x > 10)) &
    (df_pivot.groupby("Country")["NGDPRPC"].transform(lambda x: x.pct_change() < 0))
).astype(int)

# 6. Debt crisis (gross debt rising sharply above 90% of GDP)
flag_debt = (
    (df_pivot["GGXWDG_NGDP"] > 90) &
    (df_pivot.groupby("Country")["GGXWDG_NGDP"].transform(lambda x: x.diff() > 10))
).astype(int)

# 7. Fiscal crisis (large and growing deficit)
flag_fiscal = (
    (df_pivot.groupby("Country")["GGR_NGDP"].transform(lambda x: x) < 
     df_pivot.groupby("Country")["GGX_NGDP"].transform(lambda x: x) - 5) &  # Deficit > 5% GDP
    (df_pivot.groupby("Country")["GGX_NGDP"].transform(lambda x: x.diff()) > 2)  # Rising spending
).astype(int)

# 8. Current account crisis (large deficit deteriorating)
flag_current_account = (
    (df_pivot["BCA_NGDPD"] < -5) &  # Deficit > 5% of GDP
    (df_pivot.groupby("Country")["BCA_NGDPD"].transform(lambda x: x.diff() < -2))  # Worsening
).astype(int)

# 9. Real GDP growth collapse (severe contraction)
flag_growth_collapse = (
    df_pivot.groupby("Country")["NGDP_RPCH"].transform(lambda x: x < -3)  # Growth < -3%
).astype(int)

# 10. Deflation risk (falling prices with economic weakness)
flag_deflation = (
    (df_pivot.groupby("Country")["PCPIPCH"].transform(lambda x: x < 0)) &  # Negative inflation
    (df_pivot.groupby("Country")["NGDP_RPCH"].transform(lambda x: x < 1))  # Weak growth
).astype(int)

# 11. Credit crunch (investment falling while debt rising)
flag_credit_crunch = (
    (df_pivot.groupby("Country")["NID_NGDP"].transform(lambda x: x.diff() < -1)) &
    (df_pivot.groupby("Country")["GGXWDG_NGDP"].transform(lambda x: x.diff() > 5))
).astype(int)

# 12. External shock (exports collapsing while imports stable/rising)
flag_external_shock = (
    (df_pivot.groupby("Country")["TX_RPCH"].transform(lambda x: x < -5)) &
    (df_pivot.groupby("Country")["TM_RPCH"].transform(lambda x: x > -2))
).astype(int)

# --- Step 3: Combine signals into a single severity score ---
local_signal_count = (
    flag_gdp + 
    flag_invest + 
    flag_savings + 
    flag_trade + 
    flag_inflation +
    flag_debt +
    flag_fiscal +
    flag_current_account +
    flag_growth_collapse +
    flag_deflation +
    flag_credit_crunch +
    flag_external_shock
)

# --- Step 4: Multiclass recession risk label (SAME THRESHOLDS AS BEFORE) ---
def classify_risk(local_count):
    """
    Same thresholds as original:
    - 3+ signals: High risk
    - 2 signals: Moderate risk  
    - 1 signal: Mild risk
    - 0 signals: No risk
    """
    if local_count >= 3:
        return 3  # High risk
    if local_count == 2:
        return 2  # Moderate risk
    if local_count == 1:
        return 1  # Mild risk
    return 0      # No risk

df_pivot["RecessionRisk"] = [
    classify_risk(l) for l in local_signal_count
]

# Store signal count BEFORE dropna
df_pivot["SignalCount"] = local_signal_count

# --- Step 5: Clean dataset ---
df_pivot = df_pivot.dropna().sort_index(ascending=True)

# --- Step 6: Show distribution ---
print("=" * 70)
print("RECESSION RISK CLASSIFICATION SUMMARY (12 Indicators)")
print("=" * 70)

print("\nRecessionRisk class distribution:")
print(df_pivot["RecessionRisk"].value_counts().sort_index())
print(f"\nTotal samples: {len(df_pivot)}")

# Show average signals per risk class
print("\n" + "=" * 70)
print("AVERAGE SIGNALS TRIGGERED PER RISK CLASS:")
print("=" * 70)

# Now group and aggregate (SignalCount already in df_pivot)
signal_summary = df_pivot.groupby('RecessionRisk')['SignalCount'].agg(['mean', 'min', 'max', 'count'])
print(signal_summary)

# Show which signals are most common (recalculate after dropna)
print("\n" + "=" * 70)
print("INDIVIDUAL SIGNAL FREQUENCIES:")
print("=" * 70)

# Recalculate flags for cleaned data
flag_gdp_clean = (
    (df_pivot.groupby("Country")["NGDPRPC"].transform(lambda x: x.pct_change() < 0)) &
    (df_pivot.groupby("Country")["NGDPRPC"].transform(lambda x: x.pct_change().shift(-1) < 0))
).astype(int)

flag_invest_clean = (
    df_pivot.groupby("Country")["NID_NGDP"].transform(lambda x: x.diff() < -2)
).astype(int)

flag_savings_clean = (
    df_pivot.groupby("Country")["NGSD_NGDP"].transform(lambda x: x.diff() < -2)
).astype(int)

flag_trade_clean = (
    (df_pivot.groupby("Country")["TX_RPCH"].transform(lambda x: x < 0)) &
    (df_pivot.groupby("Country")["TM_RPCH"].transform(lambda x: x < 0))
).astype(int)

flag_inflation_clean = (
    (df_pivot.groupby("Country")["PCPIPCH"].transform(lambda x: x > 10)) &
    (df_pivot.groupby("Country")["NGDPRPC"].transform(lambda x: x.pct_change() < 0))
).astype(int)

flag_debt_clean = (
    (df_pivot["GGXWDG_NGDP"] > 90) &
    (df_pivot.groupby("Country")["GGXWDG_NGDP"].transform(lambda x: x.diff() > 10))
).astype(int)

flag_fiscal_clean = (
    (df_pivot.groupby("Country")["GGR_NGDP"].transform(lambda x: x) < 
     df_pivot.groupby("Country")["GGX_NGDP"].transform(lambda x: x) - 5) &
    (df_pivot.groupby("Country")["GGX_NGDP"].transform(lambda x: x.diff()) > 2)
).astype(int)

flag_current_account_clean = (
    (df_pivot["BCA_NGDPD"] < -5) &
    (df_pivot.groupby("Country")["BCA_NGDPD"].transform(lambda x: x.diff() < -2))
).astype(int)

flag_growth_collapse_clean = (
    df_pivot.groupby("Country")["NGDP_RPCH"].transform(lambda x: x < -3)
).astype(int)

flag_deflation_clean = (
    (df_pivot.groupby("Country")["PCPIPCH"].transform(lambda x: x < 0)) &
    (df_pivot.groupby("Country")["NGDP_RPCH"].transform(lambda x: x < 1))
).astype(int)

flag_credit_crunch_clean = (
    (df_pivot.groupby("Country")["NID_NGDP"].transform(lambda x: x.diff() < -1)) &
    (df_pivot.groupby("Country")["GGXWDG_NGDP"].transform(lambda x: x.diff() > 5))
).astype(int)

flag_external_shock_clean = (
    (df_pivot.groupby("Country")["TX_RPCH"].transform(lambda x: x < -5)) &
    (df_pivot.groupby("Country")["TM_RPCH"].transform(lambda x: x > -2))
).astype(int)

signal_names = [
    'GDP Decline', 'Investment Collapse', 'Savings Decline', 'Trade Shock',
    'Inflation Shock', 'Debt Crisis', 'Fiscal Crisis', 'Current Account Crisis',
    'Growth Collapse', 'Deflation Risk', 'Credit Crunch', 'External Shock'
]

signals_clean = [
    flag_gdp_clean, flag_invest_clean, flag_savings_clean, flag_trade_clean, 
    flag_inflation_clean, flag_debt_clean, flag_fiscal_clean, flag_current_account_clean,
    flag_growth_collapse_clean, flag_deflation_clean, flag_credit_crunch_clean, 
    flag_external_shock_clean
]

for name, signal in zip(signal_names, signals_clean):
    count = signal.sum()
    pct = (count / len(df_pivot)) * 100
    print(f"{name:<25} {count:>5} ({pct:>5.2f}%)")

print("\n" + "=" * 70)

# Remove temporary column
df_pivot = df_pivot.drop(columns=['SignalCount'])

# --- Output: dataframe with multiclass label ---
df_pivot

In [ ]:
# RecessionRisk Class Distribution - Compact Plot (up to 2025)
df_until_2025 = df_pivot[df_pivot.index <= 2025]
risk_counts = df_until_2025["RecessionRisk"].value_counts().sort_index()

plt.figure(figsize=(10, 6))
colors = ['green', 'yellow', 'orange', 'red']
plt.bar(risk_counts.index, risk_counts.values, color=colors, edgecolor='black', linewidth=1.5)
plt.xlabel('RecessionRisk Class', fontsize=12, fontweight='bold')
plt.ylabel('Count', fontsize=12, fontweight='bold')
plt.title('RecessionRisk Class Distribution (up to 2025)', fontsize=14, fontweight='bold')
plt.xticks([0, 1, 2, 3], ['No Risk', 'Mild', 'Moderate', 'High'])
plt.grid(axis='y', alpha=0.3)

# Add count labels on bars
for cls, count in risk_counts.items():
    plt.text(cls, count + 10, str(count), ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"Total samples (up to 2025): {len(df_until_2025)}")

In [ ]:
# All RecessionRisk Classes by Year - Line Graph (up to 2025)
# Calculate counts for each class by year
df0 = df_until_2025[df_until_2025["RecessionRisk"] == 0].copy()
df1 = df_until_2025[df_until_2025["RecessionRisk"] == 1].copy()
df2 = df_until_2025[df_until_2025["RecessionRisk"] == 2].copy()
df3 = df_until_2025[df_until_2025["RecessionRisk"] == 3].copy()

class0_by_year = df0.groupby(df0.index).size()
class1_by_year = df1.groupby(df1.index).size()
class2_by_year = df2.groupby(df2.index).size()
class3_by_year = df3.groupby(df3.index).size()

# Get all years (up to 2025)
all_years = sorted(df_until_2025.index.unique())

# Reindex to include all years (fill missing with 0)
class0_by_year = class0_by_year.reindex(all_years, fill_value=0)
class1_by_year = class1_by_year.reindex(all_years, fill_value=0)
class2_by_year = class2_by_year.reindex(all_years, fill_value=0)
class3_by_year = class3_by_year.reindex(all_years, fill_value=0)

plt.figure(figsize=(14, 7))
plt.plot(all_years, class0_by_year.values, color='green', linewidth=2.5, marker='o', label='Class 0 (No Risk)', markersize=5)
plt.plot(all_years, class1_by_year.values, color='gold', linewidth=2.5, marker='s', label='Class 1 (Mild)', markersize=5)
plt.plot(all_years, class2_by_year.values, color='orange', linewidth=2.5, marker='^', label='Class 2 (Moderate)', markersize=5)
plt.plot(all_years, class3_by_year.values, color='red', linewidth=2.5, marker='D', label='Class 3 (High)', markersize=5)

plt.xlabel('Year', fontsize=12, fontweight='bold')
plt.ylabel('Number of Countries', fontsize=12, fontweight='bold')
plt.title('RecessionRisk Classes Over Time (up to 2025)', fontsize=14, fontweight='bold')
plt.legend(fontsize=11, loc='best')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Review Remaining Countries

In [ ]:
df_filtered = df_pivot.loc[(df_pivot.index >= 2020) & (df_pivot.index <= 2025)]
df_filtered

In [ ]:
df_filtered["Country"].unique()

# 4. Exploratory Data Analysis

## Correlation Heatmap

In [ ]:
corr = df_filtered.drop(columns=["Country"]).corr()
plt.figure(figsize=(10, 8))
sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm", square=True)
plt.title("Correlation Heatmap of Features")
plt.show()

## Prepare Features and Target

In [ ]:
X = df_filtered.drop(columns=["RecessionRisk", "Country"])
y = df_filtered["RecessionRisk"]

# 5. Machine Learning Models

## Global Dataset - Full Features (13 Features)

### Define and Train All Models

In [ ]:
# ============================================================
#             TRAIN ALL MODELS (MULTICLASS)
# ============================================================
def train_all_models(X_train, y_train, X_test, y_test, model_params=None, use_xgb=False):

    if model_params is None:
        model_params = {
            'logit': {
                'C': 0.2,
                'penalty': 'l2',
                'solver': 'lbfgs',
                'max_iter': 5000,
                'random_state': 42,
                'multi_class': 'multinomial'
            },
            'rf': {
                'n_estimators': 200,
                'max_depth': 4,
                'min_samples_leaf': 20,
                'min_samples_split': 20,
                'max_features': 0.3,
                'random_state': 42
            },
            'gb': {
                'n_estimators': 200,
                'learning_rate': 0.03,
                'max_depth': 2,
                'min_samples_leaf': 20,
                'subsample': 0.6,
                'random_state': 42
            },
            'dt': {
                'max_depth': 3,
                'min_samples_leaf': 30,
                'random_state': 42
            },
            'svm': {
                'C': 1.0,
                'kernel': 'rbf',
                'probability': True,
                'random_state': 42,
                'decision_function_shape': 'ovr'
            },
            'xgb': {
                'n_estimators': 200,
                'learning_rate': 0.05,
                'max_depth': 2,
                'subsample': 0.7,
                'colsample_bytree': 0.6,
                'reg_alpha': 0.4,
                'reg_lambda': 2.0,
                'random_state': 42,
                'objective': 'multi:softprob',
                'num_class': len(np.unique(y_train))
            }
        }

    # ----------------- Train models -----------------
    logit = ImbPipeline([
        ("scaler", StandardScaler()),
        ("smote", SMOTE(random_state=42)),
        ("logit", LogisticRegression(**model_params['logit']))
    ]).fit(X_train, y_train)

    rf = ImbPipeline([
        ("smote", SMOTE(random_state=42)),
        ("rf", RandomForestClassifier(**model_params['rf']))
    ]).fit(X_train, y_train)

    gb = ImbPipeline([
        ("smote", SMOTE(random_state=42)),
        ("gb", GradientBoostingClassifier(**model_params['gb']))
    ]).fit(X_train, y_train)

    dt = ImbPipeline([
        ("smote", SMOTE(random_state=42)),
        ("dt", DecisionTreeClassifier(**model_params['dt']))
    ]).fit(X_train, y_train)

    svm = ImbPipeline([
        ("scaler", StandardScaler()),
        ("smote", SMOTE(random_state=42)),
        ("svm", SVC(**model_params['svm']))
    ]).fit(X_train, y_train)

    models = {
        "Logistic Regression": logit,
        "Random Forest": rf,
        "Gradient Boosting": gb,
        "Decision Tree": dt,
        "SVM": svm,
    }

    if use_xgb:
        xgb = ImbPipeline([
            ("smote", SMOTE(random_state=42)),
            ("xgb", XGBClassifier(**model_params['xgb']))
        ]).fit(X_train, y_train)
        models["XGBoost"] = xgb

    # ----------------- Ensemble -----------------
    ensemble_estimators = [
        ("logit", logit.named_steps["logit"]),
        ("rf", rf.named_steps["rf"]),
        ("gb", gb.named_steps["gb"]),
        ("svm", svm.named_steps["svm"])
    ]
    if use_xgb:
        ensemble_estimators.append(("xgb", xgb.named_steps["xgb"]))

    ensemble = VotingClassifier(estimators=ensemble_estimators, voting="soft")
    ensemble.fit(X_train, y_train)
    models["Ensemble"] = ensemble

    # ----------------- Metrics -----------------
    results = {}
    confusion_mats = {}

    for name, m in models.items():
        y_pred_train = m.predict(X_train)
        y_pred_test = m.predict(X_test)

        results[name] = {
            "Train Accuracy": accuracy_score(y_train, y_pred_train),
            "Test Accuracy": accuracy_score(y_test, y_pred_test),
            "Precision (macro)": precision_score(y_test, y_pred_test, average="macro", zero_division=0),
            "Recall (macro)": recall_score(y_test, y_pred_test, average="macro", zero_division=0),
            "F1 (macro)": f1_score(y_test, y_pred_test, average="macro", zero_division=0)
        }

        confusion_mats[name] = confusion_matrix(y_test, y_pred_test)

    results_df = pd.DataFrame(results).T
    return models, results_df, confusion_mats


# ============================================================
#             FEATURE IMPORTANCE (MULTICLASS)
# ============================================================
def plot_feature_importance(models, feature_names, title_prefix=""):

    logit = models.get("Logistic Regression")
    rf = models.get("Random Forest")
    gb = models.get("Gradient Boosting")
    dt = models.get("Decision Tree")

    coef_matrix = logit.named_steps['logit'].coef_
    coef_mean_abs = np.mean(np.abs(coef_matrix), axis=0)

    logit_importance = pd.DataFrame({
        "Feature": feature_names,
        "Importance": coef_mean_abs
    }).sort_values("Importance")

    rf_importance = pd.DataFrame({
        "Feature": feature_names,
        "Importance": rf.named_steps['rf'].feature_importances_
    }).sort_values("Importance")

    gb_importance = pd.DataFrame({
        "Feature": feature_names,
        "Importance": gb.named_steps['gb'].feature_importances_
    }).sort_values("Importance")

    dt_importance = pd.DataFrame({
        "Feature": feature_names,
        "Importance": dt.named_steps['dt'].feature_importances_
    }).sort_values("Importance")

    fig, axes = plt.subplots(2, 2, figsize=(16, 10))

    axes[0, 0].barh(logit_importance["Feature"], logit_importance["Importance"])
    axes[0, 0].set_title(f"{title_prefix}Logistic Regression")

    axes[0, 1].barh(rf_importance["Feature"], rf_importance["Importance"])
    axes[0, 1].set_title(f"{title_prefix}Random Forest")

    axes[1, 0].barh(gb_importance["Feature"], gb_importance["Importance"])
    axes[1, 0].set_title(f"{title_prefix}Gradient Boosting")

    axes[1, 1].barh(dt_importance["Feature"], dt_importance["Importance"])
    axes[1, 1].set_title(f"{title_prefix}Decision Tree")

    plt.tight_layout()
    plt.show()


# ============================================================
#             CONFUSION MATRIX DISPLAY (MULTICLASS)
# ============================================================
def show_confusion_matrices(confusion_mats, results_df):

    n_models = len(confusion_mats)
    fig, axes = plt.subplots(1, n_models, figsize=(5*n_models, 5))

    if n_models == 1:
        axes = [axes]

    for ax, (name, cm) in zip(axes, confusion_mats.items()):
        disp = ConfusionMatrixDisplay(confusion_matrix=cm)
        disp.plot(cmap="Blues", ax=ax, colorbar=False)

        acc = results_df.loc[name, "Test Accuracy"]
        prec = results_df.loc[name, "Precision (macro)"]
        rec = results_df.loc[name, "Recall (macro)"]
        f1 = results_df.loc[name, "F1 (macro)"]

        ax.set_title(
            f"{name}\nAcc={acc:.2f}, Prec={prec:.2f}, Rec={rec:.2f}, F1={f1:.2f}",
            fontsize=11
        )

    plt.tight_layout()
    plt.show()

# ============================================================
#             ROC CURVES + MACRO AUC (MULTICLASS)
# ============================================================
def show_roc_curves(models, X_test, y_test):
    """
    Display ROC curves and macro AUC for all models.
    Works for multiclass classification.
    """

    n_models = len(models)
    fig, axes = plt.subplots(1, n_models, figsize=(6*n_models, 5))

    if n_models == 1:
        axes = [axes]

    classes = sorted(np.unique(y_test))

    for ax, (name, model) in zip(axes, models.items()):

        if not hasattr(model, "predict_proba"):
            ax.set_title(f"{name}\n(No predict_proba)")
            ax.axis("off")
            continue

        # Predict probabilities
        y_proba = model.predict_proba(X_test)

        aucs = []
        for c in classes:
            y_true_bin = (y_test == c).astype(int)
            y_score = y_proba[:, c]

            fpr, tpr, _ = roc_curve(y_true_bin, y_score)
            auc_val = auc(fpr, tpr)
            aucs.append(auc_val)

            ax.plot(fpr, tpr, label=f"Class {c} (AUC={auc_val:.2f})")

        macro_auc = np.mean(aucs)

        ax.plot([0, 1], [0, 1], "k--", alpha=0.5)
        ax.set_title(f"{name}\nMacro AUC={macro_auc:.2f}")
        ax.set_xlabel("False Positive Rate")
        ax.set_ylabel("True Positive Rate")
        ax.legend()

    plt.tight_layout()
    plt.show()


In [ ]:
split_index = int(len(X) * 0.8)
X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]

# Training the model 

In [ ]:
models, summary_df, confusion_mats = train_all_models(X_train, y_train, X_test, y_test)

print(summary_df)
plot_feature_importance(models, X_train.columns.tolist())
# Show confusion matrices with metrics underneath
show_confusion_matrices(confusion_mats, summary_df)
show_roc_curves(models, X_test, y_test)


### Reduced Global Set

In [ ]:
#reduced variables set
selected_features = ['TM_RPCH', 'NGDP_RPCH', 'TX_RPCH', 'BCA_NGDPD']

In [ ]:
X_train_reduced = X_train[selected_features]
X_test_reduced = X_test[selected_features]

# Unpack all three return values
models_reduced, summary_df_reduced, confusion_mats_reduced = train_all_models(
    X_train_reduced, y_train, X_test_reduced, y_test
)

# Show metrics table
print(summary_df_reduced)
# Plot feature importance
plot_feature_importance(models_reduced, feature_names=selected_features, title_prefix="Reduced Features - ")

# Show confusion matrices (all in one window)
show_confusion_matrices(confusion_mats_reduced, summary_df_reduced)
show_roc_curves(models_reduced, X_test_reduced, y_test)


### Split Dataset

In [ ]:
# Map countries to continents (same logic as before)
try:
    import pycountry
    import pycountry_convert as pc
    
    def country_to_continent(name):
        try:
            lookup_name = name.replace('_', ' ')
            country = pycountry.countries.lookup(lookup_name)
            alpha2 = country.alpha_2
            cc = pc.country_alpha2_to_continent_code(alpha2)
            continent_map = {
                'AF': 'Africa',
                'AS': 'Asia',
                'EU': 'Europe',
                'NA': 'North_America',
                'OC': 'Oceania',
                'SA': 'South_America'
            }
            return continent_map.get(cc, 'Unknown')
        except Exception:
            return 'Unknown'
except ImportError:
    # Fallback mapping for common countries (extend as needed)
    fallback = {
    'United_States': 'North_America', 'Canada': 'North_America', 'Mexico': 'North_America',
    'China': 'Asia', 'India': 'Asia', 'Japan': 'Asia', 'Afghanistan': 'Asia',
    'Korea': 'Asia', 'Indonesia': 'Asia', 'Thailand': 'Asia', 'Vietnam': 'Asia',
    'Germany': 'Europe', 'France': 'Europe', 'United_Kingdom': 'Europe', 'Italy': 'Europe',
    'Spain': 'Europe', 'Russia': 'Europe', 'Turkey': 'Europe', 'Poland': 'Europe',
    'Brazil': 'South_America', 'Argentina': 'South_America', 'Chile': 'South_America',
    'Colombia': 'South_America', 'Peru': 'South_America', 'Venezuela': 'South_America',
    'Australia': 'Oceania', 'New_Zealand': 'Oceania',
    'South_Africa': 'Africa', 'Nigeria': 'Africa', 'Egypt': 'Africa', 'Zimbabwe': 'Africa',
    'Kenya': 'Africa', 'Ethiopia': 'Africa', 'Morocco': 'Africa',

    'Albania': 'Europe', 'Algeria': 'Africa', 'Austria': 'Europe', 'Barbados': 'North_America',
    'Belgium': 'Europe', 'Bolivia': 'South_America', 'Bosnia_and_Herzegovina': 'Europe',
    'Bulgaria': 'Europe', 'Cabo_Verde': 'Africa', 'Costa_Rica': 'North_America',
    'Croatia': 'Europe', 'Cyprus': 'Europe', 'Czech_Republic': 'Europe', 'Denmark': 'Europe',
    'Dominican_Republic': 'North_America', 'Estonia': 'Europe', 'Finland': 'Europe',
    'Hungary': 'Europe', 'Iceland': 'Europe', 'Ireland': 'Europe',
    'Islamic_Republic_of_Iran': 'Asia', 'Israel': 'Asia', 'Jordan': 'Asia',
    'Kazakhstan': 'Asia', 'Latvia': 'Europe', 'Lebanon': 'Asia', 'Lithuania': 'Europe',
    'Luxembourg': 'Europe', 'Malta': 'Europe', 'Netherlands': 'Europe',
    'North_Macedonia': 'Europe', 'Norway': 'Europe', 'Pakistan': 'Asia',
    'Panama': 'North_America', 'Paraguay': 'South_America', 'Portugal': 'Europe',
    'Romania': 'Europe', 'Saudi_Arabia': 'Asia', 'Serbia': 'Europe', 'Seychelles': 'Africa',
    'Slovak_Republic': 'Europe', 'Slovenia': 'Europe', 'Sweden': 'Europe',
    'Switzerland': 'Europe', 'Syria': 'Asia', 'Taiwan_Province_of_China': 'Asia',
    'Trinidad_and_Tobago': 'North_America', 'Türkiye': 'Europe', 'Uruguay': 'South_America',
    'Botswana': 'Africa', 'Cameroon': 'Africa', 'Djibouti': 'Africa', 'Equatorial_Guinea': 'Africa',
    'Eswatini': 'Africa','Guyana': 'South_America','Lesotho': 'Africa','Mali': 'Africa',
    'Mauritania': 'Africa','Namibia': 'Africa','Niger': 'Africa','Oman': 'Asia','Yemen': 'Asia',
    'Zambia': 'Africa','St._Kitts_and_Nevis': 'North_America'
}

    def country_to_continent(name):
        return fallback.get(name.replace(' ', '_'), 'Unknown')

# --- Add Continent column ---
df_filtered_copy = df_pivot.copy()
df_filtered_copy['Continent'] = df_filtered_copy['Country'].astype(str).apply(country_to_continent)

# --- Map continents to economy groups ---
continent_to_economy = {
    'Europe': 'Upper_Economies',
    'North_America': 'Upper_Economies',
    'Oceania': 'Upper_Economies',
    'Africa': 'Lower_Economies',
    'Asia': 'Lower_Economies',
    'South_America': 'Lower_Economies'
}

df_filtered_copy['EconomyGroup'] = df_filtered_copy['Continent'].map(continent_to_economy)

# --- Create Lower and Upper economy DataFrames ---
df_Lower_Economies = df_filtered_copy[df_filtered_copy['EconomyGroup'] == 'Lower_Economies'].drop(columns=['Continent','EconomyGroup'])
df_Upper_Economies = df_filtered_copy[df_filtered_copy['EconomyGroup'] == 'Upper_Economies'].drop(columns=['Continent','EconomyGroup'])

# --- Print summary ---
print("=" * 80)
print(" " * 25 + "ECONOMY GROUP SUMMARY")
print("=" * 80)

print(f"\nTotal DataFrames created:")
print(f"  • Lower Economies: {len(df_Lower_Economies)} rows")
print(f"  • Upper Economies: {len(df_Upper_Economies)} rows")
print(f"  • Total: {len(df_filtered_copy)} rows")

# --- Show RecessionRisk distribution per economy group ---
print("\n" + "=" * 80)
print("RECESSION RISK DISTRIBUTION BY ECONOMY GROUP")
print("=" * 80)

print("\n📊 LOWER ECONOMIES (Africa, Asia, South America):")
print("-" * 80)
lower_risk_dist = df_Lower_Economies['RecessionRisk'].value_counts().sort_index()
print(lower_risk_dist)
print(f"\nTotal Lower Economy samples: {len(df_Lower_Economies)}")

print("\nDetailed breakdown:")
for risk_class in range(4):
    count = lower_risk_dist.get(risk_class, 0)
    pct = (count / len(df_Lower_Economies)) * 100 if len(df_Lower_Economies) > 0 else 0
    risk_label = ['No Risk', 'Mild', 'Moderate', 'High'][risk_class]
    print(f"  Class {risk_class} ({risk_label:>8}): {count:>5} samples ({pct:>6.2f}%)")

print("\n" + "-" * 80)
print("\n📊 UPPER ECONOMIES (Europe, North America, Oceania):")
print("-" * 80)
upper_risk_dist = df_Upper_Economies['RecessionRisk'].value_counts().sort_index()
print(upper_risk_dist)
print(f"\nTotal Upper Economy samples: {len(df_Upper_Economies)}")

print("\nDetailed breakdown:")
for risk_class in range(4):
    count = upper_risk_dist.get(risk_class, 0)
    pct = (count / len(df_Upper_Economies)) * 100 if len(df_Upper_Economies) > 0 else 0
    risk_label = ['No Risk', 'Mild', 'Moderate', 'High'][risk_class]
    print(f"  Class {risk_class} ({risk_label:>8}): {count:>5} samples ({pct:>6.2f}%)")

# --- Comparative Analysis ---
print("\n" + "=" * 80)
print("COMPARATIVE ANALYSIS: LOWER vs UPPER ECONOMIES")
print("=" * 80)

print(f"\n{'Risk Class':<15} {'Lower Econ':<20} {'Upper Econ':<20} {'Difference':<15}")
print("-" * 80)

for risk_class in range(4):
    lower_count = lower_risk_dist.get(risk_class, 0)
    upper_count = upper_risk_dist.get(risk_class, 0)
    
    lower_pct = (lower_count / len(df_Lower_Economies)) * 100 if len(df_Lower_Economies) > 0 else 0
    upper_pct = (upper_count / len(df_Upper_Economies)) * 100 if len(df_Upper_Economies) > 0 else 0
    
    diff = lower_pct - upper_pct
    risk_label = ['No Risk', 'Mild', 'Moderate', 'High'][risk_class]
    
    print(f"{risk_label:<15} {lower_count:>5} ({lower_pct:>5.1f}%)     {upper_count:>5} ({upper_pct:>5.1f}%)     {diff:>+6.1f}%")

# --- Visualization ---
print("\n" + "=" * 80)
print("VISUALIZATION: Recession Risk by Economy Group")
print("=" * 80)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Lower Economies
lower_counts = [lower_risk_dist.get(i, 0) for i in range(4)]
axes[0].bar(range(4), lower_counts, color=['green', 'yellow', 'orange', 'red'], 
            edgecolor='black', linewidth=1.5)
axes[0].set_xlabel('RecessionRisk Class', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Count', fontsize=12, fontweight='bold')
axes[0].set_title('Lower Economies\n(Africa, Asia, South America)', fontsize=14, fontweight='bold')
axes[0].set_xticks(range(4))
axes[0].set_xticklabels(['No Risk', 'Mild', 'Moderate', 'High'])
axes[0].grid(axis='y', alpha=0.3)

# Add count labels
for i, count in enumerate(lower_counts):
    if count > 0:
        axes[0].text(i, count + max(lower_counts)*0.02, str(count), 
                    ha='center', va='bottom', fontsize=11, fontweight='bold')

# Upper Economies
upper_counts = [upper_risk_dist.get(i, 0) for i in range(4)]
axes[1].bar(range(4), upper_counts, color=['green', 'yellow', 'orange', 'red'], 
            edgecolor='black', linewidth=1.5)
axes[1].set_xlabel('RecessionRisk Class', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Count', fontsize=12, fontweight='bold')
axes[1].set_title('Upper Economies\n(Europe, North America, Oceania)', fontsize=14, fontweight='bold')
axes[1].set_xticks(range(4))
axes[1].set_xticklabels(['No Risk', 'Mild', 'Moderate', 'High'])
axes[1].grid(axis='y', alpha=0.3)

# Add count labels
for i, count in enumerate(upper_counts):
    if count > 0:
        axes[1].text(i, count + max(upper_counts)*0.02, str(count), 
                    ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

# --- Countries per economy group ---
print("\n" + "=" * 80)
print("COUNTRIES BY ECONOMY GROUP")
print("=" * 80)

print("\nLower Economies Countries:")
lower_countries = sorted(df_Lower_Economies['Country'].unique())
print(f"  Total: {len(lower_countries)} countries")
print(f"  {', '.join(lower_countries[:10])}...")  # Show first 10

print("\nUpper Economies Countries:")
upper_countries = sorted(df_Upper_Economies['Country'].unique())
print(f"  Total: {len(upper_countries)} countries")
print(f"  {', '.join(upper_countries[:10])}...")  # Show first 10

print("\n" + "=" * 80)

# 6. Economy-Specific Analysis

## Upper Economies - Full Features

In [ ]:
# Prepare data
X = df_Upper_Economies.drop(columns=["RecessionRisk", "Country"])
y = df_Upper_Economies["RecessionRisk"]

split_index = int(len(X) * 0.8)
X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]

# Train models and unpack all three return values
models_upper, summary_df_upper, confusion_mats_upper = train_all_models(
    X_train, y_train, X_test, y_test
)

# Show metrics table
print(summary_df_upper)

# Plot feature importance
plot_feature_importance(models_upper, X_train.columns.tolist(), title_prefix="Upper Economies - ")

# Show confusion matrices (all in one window)
show_confusion_matrices(confusion_mats_upper, summary_df_upper)
# ✅ Show ROC curves + AUC for upper-economy models
show_roc_curves(models_upper, X_test, y_test)

## Lower Economies - Full Features

In [ ]:
# Prepare data for Lower Economies
X = df_Lower_Economies.drop(columns=["RecessionRisk", "Country"])
y = df_Lower_Economies["RecessionRisk"]

split_index = int(len(X) * 0.8)
X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]

# Train models and unpack all three return values
models_lower, summary_df_lower, confusion_mats_lower = train_all_models(
    X_train, y_train, X_test, y_test
)

# Show metrics table
print(summary_df_lower)

# Plot feature importance
plot_feature_importance(models_lower, X_train.columns.tolist(), title_prefix="Lower Economies - ")


# Show confusion matrices (all in one window)
show_confusion_matrices(confusion_mats_lower, summary_df_lower)
# ✅ Show ROC curves + AUC for lower-economy models
show_roc_curves(models_lower, X_test, y_test)

## Upper Economies - Reduced Features

In [ ]:
X_upper = df_Upper_Economies[selected_features]
y_upper = df_Upper_Economies["RecessionRisk"]

split_index_upper = int(len(X_upper) * 0.8)
X_train_upper = X_upper.iloc[:split_index_upper]
X_test_upper = X_upper.iloc[split_index_upper:]
y_train_upper = y_upper.iloc[:split_index_upper]
y_test_upper = y_upper.iloc[split_index_upper:]

# ✅ Unpack all three return values
models_upper_reduced, summary_df_upper_reduced, confusion_mats_upper_reduced = train_all_models(
    X_train_upper, y_train_upper, X_test_upper, y_test_upper
)

print("Upper Economies Accuracy (Reduced Features):")
print(summary_df_upper_reduced)

# Feature importance
plot_feature_importance(
    models_upper_reduced,
    feature_names=selected_features,
    title_prefix="Upper Economies - Reduced Features - "
)

# ✅ Show confusion matrices (all in one window)
show_confusion_matrices(confusion_mats_upper_reduced, summary_df_upper_reduced)

# ✅ Show ROC curves + AUC for reduced-feature upper-economy models
show_roc_curves(models_upper_reduced, X_test_upper, y_test_upper)


## Lower Economies - Reduced Features

In [ ]:
X_lower = df_Lower_Economies[selected_features]
y_lower = df_Lower_Economies["RecessionRisk"]

split_index_lower = int(len(X_lower) * 0.8)
X_train_lower = X_lower.iloc[:split_index_lower]
X_test_lower = X_lower.iloc[split_index_lower:]
y_train_lower = y_lower.iloc[:split_index_lower]
y_test_lower = y_lower.iloc[split_index_lower:]

# ✅ Unpack all three return values
models_lower_reduced, summary_df_lower_reduced, confusion_mats_lower_reduced = train_all_models(
    X_train_lower, y_train_lower, X_test_lower, y_test_lower
)

print("Lower Economies Accuracy (Reduced Features):")
print(summary_df_lower_reduced)

# Feature importance
plot_feature_importance(
    models_lower_reduced,
    feature_names=selected_features,
    title_prefix="Lower Economies - Reduced Features - "
)

# ✅ Show confusion matrices (all in one window)
show_confusion_matrices(confusion_mats_lower_reduced, summary_df_lower_reduced)

# ✅ Show ROC curves + AUC for reduced-feature lower-economy models
show_roc_curves(models_lower_reduced, X_test_lower, y_test_lower)


# Prediction 2026-2030

In [ ]:
df_predict = df_pivot.loc[df_pivot.index > 2025]
df_predict_original = df_predict.copy()
df_predict = df_predict.drop(columns=["RecessionRisk", "Country"])

df_predict_original['Continent'] = df_predict_original['Country'].astype(str).apply(country_to_continent)

continent_to_economy = {
    'Europe': 'Upper_Economies',
    'North_America': 'Upper_Economies',
    'Oceania': 'Upper_Economies',
    'Africa': 'Lower_Economies',
    'Asia': 'Lower_Economies',
    'South_America': 'Lower_Economies'
}

df_predict_original['EconomyGroup'] = df_predict_original['Continent'].map(continent_to_economy)

df_predict_lower = df_predict_original[df_predict_original['EconomyGroup'] == 'Lower_Economies']

df_predict_upper = df_predict_original[df_predict_original['EconomyGroup'] == 'Upper_Economies']

print("Created economy-specific prediction DataFrames from df_predict_original:")
print(f" - Lower_Economies predictions: {len(df_predict_lower)} rows")
print(f" - Upper_Economies predictions: {len(df_predict_upper)} rows")

In [ ]:
# ============================================================
#                  PREDICTION FUNCTION (MULTICLASS)
# ============================================================
def make_predictions(models, df_predict):
    """
    Return multiclass predictions (0,1,2,3) from every model in one dataframe.
    """
    predictions = {}

    for name, model in models.items():
        # If model supports predict_proba → use argmax over classes
        if hasattr(model, "predict_proba"):
            proba = model.predict_proba(df_predict)
            predictions[name] = proba.argmax(axis=1)
        else:
            predictions[name] = model.predict(df_predict)

    return pd.DataFrame(predictions, index=df_predict.index)


# ============================================================
#      MULTICLASS RECESSION RISK COUNTS PER MODEL (PLOT)
# ============================================================
def plot_recession_counts_per_model(df_with_country, title):
    """
    Grouped bar plot per year per model:
    - X: Year
    - Bars: counts of each RecessionRisk class (0,1,2,3)
    - Facets: one subplot per model

    Expects df_with_country with columns:
        ['Year','Country', <model prediction columns>]
    where model columns contain multiclass predictions (0–3).
    """

    # Melt to long format
    df_long = df_with_country.melt(
        id_vars=['Year', 'Country'],
        var_name='Model',
        value_name='Prediction'
    )

    # Count per (Year, Model, Prediction)
    counts = (
        df_long.groupby(['Year', 'Model', 'Prediction'])
               .size()
               .reset_index(name='Count')
    )

    # Pivot → columns become risk classes (0,1,2,3)
    counts_pivot = (
        counts.pivot(index=['Year', 'Model'], columns='Prediction', values='Count')
              .fillna(0)
    ).reset_index()

    # Identify risk classes present
    risk_classes = sorted([c for c in counts_pivot.columns if isinstance(c, int)])

    # Reorder columns: Year, Model, then risk classes
    counts_pivot = counts_pivot[['Year', 'Model'] + risk_classes]

    # Plotting layout
    models = counts_pivot['Model'].unique()
    n_models = len(models)
    n_cols = 3
    n_rows = (n_models + n_cols - 1) // n_cols

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(5 * n_cols, 4 * n_rows), sharex=False, sharey=True)
    axes = axes.flatten()

    for i, model in enumerate(models):
        ax = axes[i]

        # Extract model-specific data
        mdf = counts_pivot[counts_pivot['Model'] == model].set_index('Year')[risk_classes]

        # Plot grouped bars
        plot = mdf.plot(kind='bar', ax=ax)

        ax.set_title(model)
        ax.set_ylabel('Number of countries')
        ax.set_xlabel('Year')
        ax.legend(title='RecessionRisk')

        # Annotate bars
        for p in plot.patches:
            height = p.get_height()
            if height > 0:
                ax.annotate(
                    f'{int(height)}',
                    (p.get_x() + p.get_width() / 2., height),
                    ha='center', va='bottom',
                    fontsize=8, color='black', xytext=(0, 2),
                    textcoords='offset points'
                )

    # Hide unused axes
    for j in range(i + 1, len(axes)):
        axes[j].set_visible(False)

    fig.suptitle(title, y=1.02, fontsize=12)
    fig.tight_layout()
    plt.show()


In [ ]:
df_predict_original.drop(columns=["RecessionRisk"], inplace=True)
df_predict_original

In [ ]:
df_predict_upper.drop(columns=["RecessionRisk"], inplace=True)
df_predict_upper

In [ ]:
df_predict_lower.drop(columns=["RecessionRisk"], inplace=True)
df_predict_lower

In [ ]:
df_predict_upper_copy = df_predict_upper.copy()
df_predict_lower_copy = df_predict_lower.copy()
df_predict_upper_copy.drop(columns=['Continent', 'EconomyGroup', 'Country'], inplace=True)
df_predict_lower_copy.drop(columns=['Continent', 'EconomyGroup', 'Country'], inplace=True)

In [ ]:
# Reset index once for reuse (assumes Year is the index in df_predict_original)
df_predict_original_reset = df_predict_original.reset_index()
df_predict_upper = df_predict_upper.reset_index()
df_predict_lower = df_predict_lower.reset_index()

In [ ]:
# ============================================================
# Global (all countries, full feature set)
# ============================================================
predictions = make_predictions(models, df_predict)
predictions_global_features_with_country = pd.concat(
    [df_predict_original_reset[['Year', 'Country']], predictions.reset_index(drop=True)],
    axis=1
)
print(predictions_global_features_with_country.head())
plot_recession_counts_per_model(predictions_global_features_with_country, "Global (Full features)")


In [ ]:
# ============================================================
# Global Restricted (all countries, restricted feature set)
# ============================================================
df_predict_restricted = df_predict[selected_features]
predictions_restricted_features = make_predictions(models_reduced, df_predict_restricted)
predictions_global_restricted_features_with_country = pd.concat(
    [df_predict_original_reset[['Year', 'Country']], predictions_restricted_features.reset_index(drop=True)],
    axis=1
)
print(predictions_global_restricted_features_with_country.head())
plot_recession_counts_per_model(predictions_global_restricted_features_with_country, "Global (Restricted features)")

In [ ]:

# ============================================================
# Upper Economies (full feature set)
# ============================================================
df_predict_upper_reset = df_predict_upper_copy.reset_index(drop=True)

X_predict_upper = df_predict_upper_reset.drop(columns=["RecessionRisk", "Country"], errors="ignore")
predictions_upper_features = make_predictions(models_upper, X_predict_upper)

predictions_upper_features_with_country = pd.concat(
    [df_predict_upper[["Year", "Country"]], predictions_upper_features.reset_index(drop=True)],
    axis=1
)

print(predictions_upper_features_with_country.head())
plot_recession_counts_per_model(predictions_upper_features_with_country, "Upper economies (Full features)")

In [ ]:
# ============================================================
# Lower Economies (full feature set)
# ============================================================
df_predict_lower_reset = df_predict_lower_copy.reset_index(drop=True)

X_predict_lower = df_predict_lower_reset.drop(columns=["RecessionRisk", "Country"], errors="ignore")
predictions_lower_features = make_predictions(models_lower, X_predict_lower)

predictions_lower_features_with_country = pd.concat(
    [df_predict_lower[["Year", "Country"]], predictions_lower_features.reset_index(drop=True)],
    axis=1
)

print(predictions_lower_features_with_country.head())
plot_recession_counts_per_model(predictions_lower_features_with_country, "Lower economies (Full features)")

In [ ]:
# ============================================================
# Upper Economies (restricted feature set)
# ============================================================
X_predict_upper_reduced = df_predict_upper[selected_features]
df_predict_upper_reset = df_predict_upper_copy.reset_index(drop=True)

X_predict_upper_reduced = df_predict_upper_reset[selected_features]
predictions_upper_restricted_features = make_predictions(models_upper_reduced, X_predict_upper_reduced)

predictions_upper_restricted_features_with_country = pd.concat(
    [df_predict_upper[["Year", "Country"]], predictions_upper_restricted_features.reset_index(drop=True)],
    axis=1
)

print(predictions_upper_restricted_features_with_country.head())
plot_recession_counts_per_model(predictions_upper_restricted_features_with_country, "Upper economies (Restricted features)")

In [ ]:
# ============================================================
# Lower Economies (restricted feature set)
# ============================================================
X_predict_lower_reduced = df_predict_lower[selected_features]
df_predict_lower_reset = df_predict_lower_copy.reset_index(drop=True)
X_predict_lower_reduced = df_predict_lower_reset[selected_features]
predictions_lower_restricted_features = make_predictions(models_lower_reduced, X_predict_lower_reduced)

predictions_lower_restricted_features_with_country = pd.concat(
    [df_predict_lower[["Year", "Country"]], predictions_lower_restricted_features.reset_index(drop=True)],
    axis=1
)

print(predictions_lower_restricted_features_with_country.head())
plot_recession_counts_per_model(predictions_lower_restricted_features_with_country, "Lower economies (Restricted features)")

# LTSM Neural Networks

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# Core layers
from tensorflow.keras.layers import (
    Input,
    Dense,
    LSTM,
    GRU,
    Dropout,
    BatchNormalization,
    LayerNormalization,
    Bidirectional
)

# Advanced sequence layers
from tensorflow.keras.layers import (
    Conv1D,
    GlobalAveragePooling1D,
    GlobalMaxPooling1D,
    MultiHeadAttention
)

# Model utilities
from tensorflow.keras.models import Sequential, Model

# Callbacks
from tensorflow.keras.callbacks import (
    EarlyStopping,
    ReduceLROnPlateau,
    ModelCheckpoint
)

# Optimizers
from tensorflow.keras.optimizers import Adam, RMSprop

# Losses & metrics
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy


In [ ]:
df_until_2025 = df_pivot[df_pivot.index <= 2025]
df_after_2025 = df_pivot[df_pivot.index > 2025]


In [ ]:
df_until_2025

In [ ]:
df_after_2025

In [ ]:
def reshape_for_lstm(df, window=5, min_sequences=1, return_feature_names=False):
    """
    Convert df_pivot (Year index, Country column) into LSTM-ready sequences.
    
    Args:
        df: DataFrame with Year index and Country column
        window: Number of historical years to use as input
        min_sequences: Minimum sequences required per country (default 1)
        return_feature_names: If True, return feature column names
        
    Returns:
        X_seq: (num_sequences, window, num_features)
        y_seq: (num_sequences,)
        countries_seq: country for each sequence
        years_seq: ending year for each sequence
        feature_names: (optional) list of feature column names
    """
    df = df.copy()
    
    # Handle index - reset if needed for sorting
    if isinstance(df.index, pd.RangeIndex):
        raise ValueError("DataFrame must have Year as index")
    
    # Reset index to make sorting easier, then sort
    df_sorted = df.reset_index().sort_values(["Country", df.index.name])
    year_col = df.index.name or "Year"
    
    # Define features (exclude target and identifiers)
    feature_cols = [c for c in df_sorted.columns 
                   if c not in ["RecessionRisk", "Country", year_col]]
    
    X_seq = []
    y_seq = []
    countries_seq = []
    years_seq = []
    
    # Group by country
    for country, group in df_sorted.groupby("Country"):
        group = group.sort_values(year_col).reset_index(drop=True)
        
        # Check if country has enough data
        if len(group) < window + 1:
            print(f"Warning: {country} has only {len(group)} years, need {window + 1}. Skipping.")
            continue
        
        X_values = group[feature_cols].values
        y_values = group["RecessionRisk"].values
        years = group[year_col].values
        
        # Sliding window
        num_sequences = len(group) - window
        for i in range(num_sequences):
            X_seq.append(X_values[i:i+window])
            y_seq.append(y_values[i+window])
            countries_seq.append(country)
            years_seq.append(years[i+window])
    
    if len(X_seq) < min_sequences:
        raise ValueError(f"Only generated {len(X_seq)} sequences, need at least {min_sequences}")
    
    results = (
        np.array(X_seq),
        np.array(y_seq),
        np.array(countries_seq),
        np.array(years_seq)
    )
    
    if return_feature_names:
        results = results + (feature_cols,)
    
    return results

In [ ]:
WINDOW = 5

X_seq, y_seq, countries_seq, years_seq = reshape_for_lstm(
    df_until_2025,
    window=WINDOW
)

print("X_seq shape:", X_seq.shape)
print("y_seq shape:", y_seq.shape)
print("Example sequence shape:", X_seq[0].shape)


In [ ]:
# Option 1: Simple time-based split (most common for time series)
# Train on earlier years, test on most recent years
SPLIT_YEAR = 2019  # Adjust based on your data

train_mask = years_seq <= SPLIT_YEAR
test_mask = years_seq > SPLIT_YEAR

# This gives you:
# Train: all sequences predicting up to 2019
# Test: sequences predicting 2020,2021,2022,2023, 2024, 2025

X_train, X_test = X_seq[train_mask], X_seq[test_mask]
y_train, y_test = y_seq[train_mask], y_seq[test_mask]
countries_train = countries_seq[train_mask]
countries_test  = countries_seq[test_mask]
years_train = years_seq[train_mask]
years_test  = years_seq[test_mask]

print(f"Train sequences: {len(X_train)} (predicting years {years_train.min()}-{years_train.max()})")
print(f"Test sequences: {len(X_test)} (predicting years {years_test.min()}-{years_test.max()})")
print(f"Train: {np.sum(train_mask)} sequences, Test: {np.sum(test_mask)} sequences")
print(f"Split: {np.sum(train_mask)/(np.sum(train_mask)+np.sum(test_mask))*100:.1f}% train, {np.sum(test_mask)/(np.sum(train_mask)+np.sum(test_mask))*100:.1f}% test")

In [ ]:
# ADVANCED LSTM MODEL - MAXIMUM PERFORMANCE

# Step 1: Scale the features
X_train_reshaped = X_train.reshape(-1, X_train.shape[-1])
X_test_reshaped = X_test.reshape(-1, X_test.shape[-1])

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_reshaped)
X_test_scaled = scaler.transform(X_test_reshaped)

X_train_scaled = X_train_scaled.reshape(X_train.shape)
X_test_scaled = X_test_scaled.reshape(X_test.shape)

print("Data scaled successfully")
print(f"X_train_scaled shape: {X_train_scaled.shape}")
print(f"X_test_scaled shape: {X_test_scaled.shape}")
print("\n" + "="*70 + "\n")

# Step 2: Check class distribution
print("Unique RecessionRisk classes:", np.unique(y_seq))
print("\nClass distribution in train:")
train_dist = pd.Series(y_train).value_counts().sort_index()
print(train_dist)
print("\nClass distribution in test:")
test_dist = pd.Series(y_test).value_counts().sort_index()
print(test_dist)

num_classes = len(np.unique(y_seq))
print(f"\nNumber of classes: {num_classes}")
print("\n" + "="*70 + "\n")

# Step 3: SMOTE-like oversampling for training data (synthetic minority samples)
from imblearn.over_sampling import SMOTE

# Flatten for SMOTE
X_train_flat = X_train_scaled.reshape(X_train_scaled.shape[0], -1)

# Apply SMOTE
print("Applying SMOTE to balance training data...")
smote = SMOTE(random_state=42, k_neighbors=3)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_flat, y_train)

# Reshape back
X_train_resampled = X_train_resampled.reshape(-1, WINDOW, X_train_scaled.shape[2])

print(f"Original training size: {len(X_train_scaled)}")
print(f"Resampled training size: {len(X_train_resampled)}")
print("\nResampled class distribution:")
print(pd.Series(y_train_resampled).value_counts().sort_index())
print("\n" + "="*70 + "\n")

# Step 4: Build deeper model with attention mechanism
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, GlobalAveragePooling1D

# Custom attention-enhanced LSTM model
inputs = keras.Input(shape=(WINDOW, X_train_scaled.shape[2]))

# First bidirectional LSTM
x = keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=True))(inputs)
x = keras.layers.LayerNormalization()(x)
x = keras.layers.Dropout(0.3)(x)

# Second bidirectional LSTM
x = keras.layers.Bidirectional(keras.layers.LSTM(64, return_sequences=True))(x)
x = keras.layers.LayerNormalization()(x)
x = keras.layers.Dropout(0.3)(x)

# Attention layer
attention = keras.layers.MultiHeadAttention(num_heads=4, key_dim=32)(x, x)
attention = keras.layers.LayerNormalization()(attention)
x = keras.layers.Add()([x, attention])  # Residual connection

# Global pooling
x = keras.layers.GlobalAveragePooling1D()(x)

# Dense layers
x = keras.layers.Dense(128, activation="relu")(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.4)(x)

x = keras.layers.Dense(64, activation="relu")(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dropout(0.3)(x)

x = keras.layers.Dense(32, activation="relu")(x)
x = keras.layers.Dropout(0.2)(x)

# Output
outputs = keras.layers.Dense(num_classes, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()
print("\n" + "="*70 + "\n")

# Step 5: Custom focal loss (properly implemented)
class FocalLoss(keras.losses.Loss):
    def __init__(self, gamma=2.0, alpha=None):
        super().__init__()
        self.gamma = gamma
        self.alpha = alpha
        
    def call(self, y_true, y_pred):
        y_true = tf.cast(y_true, tf.int32)
        y_true = tf.reshape(y_true, [-1])
        y_true_onehot = tf.one_hot(y_true, depth=num_classes)
        
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1.0 - epsilon)
        
        cross_entropy = -y_true_onehot * tf.math.log(y_pred)
        weight = tf.math.pow(1.0 - y_pred, self.gamma)
        
        if self.alpha is not None:
            alpha_t = tf.gather(self.alpha, y_true)
            alpha_t = tf.reshape(alpha_t, [-1, 1])
            weight = weight * alpha_t
        
        focal_loss = weight * cross_entropy
        return tf.reduce_mean(tf.reduce_sum(focal_loss, axis=-1))

# Calculate class weights for focal loss
total_samples = len(y_train_resampled)
class_counts = np.bincount(y_train_resampled)
alpha_weights = tf.constant([
    np.sqrt(total_samples / (num_classes * count)) for count in class_counts
], dtype=tf.float32)

# Step 6: Compile with focal loss
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss=FocalLoss(gamma=2.0, alpha=alpha_weights),
    metrics=["accuracy"]
)

print("Model compiled with Focal Loss")
print("\n" + "="*70 + "\n")

# Step 7: Advanced callbacks
from tensorflow.keras.callbacks import LearningRateScheduler

def scheduler(epoch, lr):
    if epoch < 30:
        return lr
    elif epoch < 60:
        return lr * 0.5
    elif epoch < 90:
        return lr * 0.1
    else:
        return lr * 0.01

callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=25,
        restore_best_weights=True,
        verbose=1
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=10,
        min_lr=1e-7,
        verbose=1
    ),
    LearningRateScheduler(scheduler, verbose=0)
]

# Step 8: Train with resampled data
history = model.fit(
    X_train_resampled, y_train_resampled,
    validation_data=(X_test_scaled, y_test),
    epochs=150,
    batch_size=32,
    shuffle=True,  # Now we can shuffle because SMOTE created synthetic samples
    callbacks=callbacks,
    verbose=1
)

print("\n" + "="*70 + "\n")

# Step 9: Evaluate
test_loss, test_acc = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")
print("\n" + "="*70 + "\n")

# Step 10: Generate predictions
y_pred = model.predict(X_test_scaled)
y_pred_classes = np.argmax(y_pred, axis=1)

# Step 11: Calculate metrics
precision = precision_score(y_test, y_pred_classes, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred_classes, average='weighted', zero_division=0)
f1 = f1_score(y_test, y_pred_classes, average='weighted', zero_division=0)
macro_f1 = f1_score(y_test, y_pred_classes, average='macro', zero_division=0)

print(f"Test Precision (weighted): {precision:.4f}")
print(f"Test Recall (weighted): {recall:.4f}")
print(f"Test F1-Score (weighted): {f1:.4f}")
print(f"Test F1-Score (macro): {macro_f1:.4f}")
print("\n" + "="*70 + "\n")

# Step 12: Per-class performance
print("Per-class performance:")
print("-" * 70)
for cls in range(num_classes):
    cls_mask = y_test == cls
    if np.sum(cls_mask) > 0:
        cls_acc = np.mean(y_pred_classes[cls_mask] == cls)
        cls_count = np.sum(cls_mask)
        predicted_as_cls = np.sum(y_pred_classes == cls)
        
        cls_precision = precision_score(y_test == cls, y_pred_classes == cls, zero_division=0)
        cls_recall = recall_score(y_test == cls, y_pred_classes == cls, zero_division=0)
        cls_f1 = f1_score(y_test == cls, y_pred_classes == cls, zero_division=0)
        
        print(f"Class {cls}:")
        print(f"  Actual: {cls_count:>3} | Predicted: {predicted_as_cls:>3} | Acc: {cls_acc:.3f} | P: {cls_precision:.3f} | R: {cls_recall:.3f} | F1: {cls_f1:.3f}")

print("\n" + "="*70 + "\n")

# Step 13: Confusion matrix
cm = confusion_matrix(y_test, y_pred_classes)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Absolute numbers
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax1,
            xticklabels=[f'Class {i}' for i in range(num_classes)],
            yticklabels=[f'Class {i}' for i in range(num_classes)])
ax1.set_xlabel('Predicted')
ax1.set_ylabel('Actual')
ax1.set_title('Confusion Matrix (Counts)')

# Percentages
cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
sns.heatmap(cm_percent, annot=True, fmt='.1f', cmap='Blues', ax=ax2,
            xticklabels=[f'Class {i}' for i in range(num_classes)],
            yticklabels=[f'Class {i}' for i in range(num_classes)])
ax2.set_xlabel('Predicted')
ax2.set_ylabel('Actual')
ax2.set_title('Confusion Matrix (Percentages)')

plt.tight_layout()
plt.show()

# Step 14: Classification report
print("Classification Report:")
print("=" * 70)
print(classification_report(y_test, y_pred_classes, 
                          target_names=[f'Class {i}' for i in range(num_classes)],
                          zero_division=0))

# Step 15: Plot training history
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].plot(history.history['loss'], label='Train Loss', linewidth=2)
axes[0].plot(history.history['val_loss'], label='Val Loss', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss', fontsize=12)
axes[0].legend(fontsize=11)
axes[0].set_title('Training and Validation Loss', fontsize=14)
axes[0].grid(True, alpha=0.3)

axes[1].plot(history.history['accuracy'], label='Train Accuracy', linewidth=2)
axes[1].plot(history.history['val_accuracy'], label='Val Accuracy', linewidth=2)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Accuracy', fontsize=12)
axes[1].legend(fontsize=11)
axes[1].set_title('Training and Validation Accuracy', fontsize=14)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Step 16: Prediction distribution
print("\nPrediction Distribution Analysis:")
print("-" * 70)
print(f"{'Class':<10} {'Actual':<15} {'Predicted':<15} {'Difference':<15}")
print("-" * 70)
for cls in range(num_classes):
    actual = np.sum(y_test == cls)
    predicted = np.sum(y_pred_classes == cls)
    diff = predicted - actual
    actual_pct = actual / len(y_test) * 100
    pred_pct = predicted / len(y_pred_classes) * 100
    print(f"Class {cls}    {actual} ({actual_pct:>5.1f}%)    {predicted} ({pred_pct:>5.1f}%)    {diff:+4d} ({pred_pct-actual_pct:+5.1f}%)")

# Step 17: Confidence analysis
print("\n" + "="*70)
print("\nPrediction Confidence Analysis:")
print("-" * 70)

correct_mask = y_pred_classes == y_test
if np.sum(correct_mask) > 0:
    print(f"Correct predictions   (n={np.sum(correct_mask):>3}): avg conf = {np.mean(np.max(y_pred[correct_mask], axis=1)):.3f}")
if np.sum(~correct_mask) > 0:
    print(f"Incorrect predictions (n={np.sum(~correct_mask):>3}): avg conf = {np.mean(np.max(y_pred[~correct_mask], axis=1)):.3f}")

print("\nPer-class confidence:")
for cls in range(num_classes):
    cls_pred_mask = y_pred_classes == cls
    if np.sum(cls_pred_mask) > 0:
        cls_confidences = np.max(y_pred[cls_pred_mask], axis=1)
        print(f"Class {cls}: n={np.sum(cls_pred_mask):>3}, avg={np.mean(cls_confidences):.3f}, min={np.min(cls_confidences):.3f}, max={np.max(cls_confidences):.3f}")